<a href="https://colab.research.google.com/github/safaeOulaja/Embedding-Search-NTCIR-CLEF-TREC/blob/main/Embedding%2BSearch_TREC_DATASET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Downloading the ClinicalTrials dataset
!wget https://www.trec-cds.org/2021_data/ClinicalTrials.2021-04-27.part1.zip -O ClinicalTrials.zip

!unzip -o ClinicalTrials.zip -d /content/

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: /content/ClinicalTrials.2021-04-27.part1/NCT0093xxxx/NCT00934219.xml  
  inflating: /content/ClinicalTrials.2021-04-27.part1/NCT0093xxxx/NCT00934232.xml  
  inflating: /content/ClinicalTrials.2021-04-27.part1/NCT0093xxxx/NCT00934245.xml  
  inflating: /content/ClinicalTrials.2021-04-27.part1/NCT0093xxxx/NCT00934258.xml  
  inflating: /content/ClinicalTrials.2021-04-27.part1/NCT0093xxxx/NCT00934271.xml  
  inflating: /content/ClinicalTrials.2021-04-27.part1/NCT0093xxxx/NCT00934284.xml  
  inflating: /content/ClinicalTrials.2021-04-27.part1/NCT0093xxxx/NCT00934297.xml  
  inflating: /content/ClinicalTrials.2021-04-27.part1/NCT0093xxxx/NCT00934310.xml  
  inflating: /content/ClinicalTrials.2021-04-27.part1/NCT0093xxxx/NCT00934323.xml  
  inflating: /content/ClinicalTrials.2021-04-27.part1/NCT0093xxxx/NCT00934336.xml  
  inflating: /content/ClinicalTrials.2021-04-27.part1/NCT0093xxxx/NCT00934349.xml 

In [ ]:
# Navigating an example of the xml file
!cat /content/ClinicalTrials.2021-04-27.part1/NCT0000xxxx/NCT00000102.xml

<clinical_study>
  <!-- This xml conforms to an XML Schema at:
    https://clinicaltrials.gov/ct2/html/images/info/public.xsd -->
  <required_header>
    <download_date>ClinicalTrials.gov processed this data on April 27, 2021</download_date>
    <link_text>Link to the current ClinicalTrials.gov record.</link_text>
    <url>https://clinicaltrials.gov/show/NCT00000102</url>
  </required_header>
  <id_info>
    <org_study_id>NCRR-M01RR01070-0506</org_study_id>
    <secondary_id>M01RR001070</secondary_id>
    <nct_id>NCT00000102</nct_id>
  </id_info>
  <brief_title>Congenital Adrenal Hyperplasia: Calcium Channels as Therapeutic Targets</brief_title>
  <sponsors>
    <lead_sponsor>
      <agency>National Center for Research Resources (NCRR)</agency>
      <agency_class>NIH</agency_class>
    </lead_sponsor>
  </sponsors>
  <source>National Center for Research Resources (NCRR)</source>
  <brief_summary>
    <textblock>
      This study will test the ability of extended release nifedipine (Pr

In [ ]:
# necessary imports
!pip install sentence-transformers

import os
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from lxml import etree

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# Defining a function to parse XML files and extract relevant information
def parse_clinical_trials_data(directory):
    data = []

    # Walk through all files in the directory
    for foldername in os.listdir(directory):
        folderpath = os.path.join(directory, foldername)
        if os.path.isdir(folderpath):
            for filename in os.listdir(folderpath):
                file_path = os.path.join(folderpath, filename)
                if file_path.endswith(".xml"):
                    # Parse the XML file
                    tree = etree.parse(file_path)
                    root = tree.getroot()

                    # Extract the relevant information
                    brief_title = root.findtext('.//brief_title', default="N/A")
                    brief_summary = root.findtext('.//brief_summary//textblock', default="N/A")
                    detailed_description = root.findtext('.//detailed_description//textblock', default="N/A")

                    # Combine the text fields into a single field
                    combined_text = f"{brief_title}. {brief_summary}. {detailed_description}"

                    # Append the data to the list
                    data.append({
                        'NCT_ID': root.findtext('.//id_info//nct_id', default="N/A"),
                        'Title': brief_title,
                        'Brief_Summary': brief_summary,
                        'Detailed_Description': detailed_description,
                        'Combined': combined_text
                    })

    # Convert the data into a DataFrame
    df = pd.DataFrame(data)
    return df

In [ ]:
# Parsing the data from XML files into a DataFrame
directory = '/content/ClinicalTrials.2021-04-27.part1'
df = parse_clinical_trials_data(directory)

In [ ]:
# Loading the Sentence Transformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Generate embeddings for the 'Combined' text in the DataFrame
df['Embedding_Vector'] = df['Combined'].apply(lambda x: model.encode(x))

In [ ]:
def search_review(model, df, query, n=3):
    # Generate the embedding vector for the query sentence
    query_embedding = model.encode([query])
    query_embedding = query_embedding.reshape(1, -1)  # Ensure it's 2D

    # Calculate cosine similarity between the query embedding and each embedding in 'Embedding_Vector'
    df['Cosine_Similarity'] = df['Embedding_Vector'].apply(
        lambda x: cosine_similarity([query_embedding[0]], [x])[0][0]
    )

    # Sort by cosine similarity in descending order and return the top 'n' results
    top_n_df = df.sort_values(by='Cosine_Similarity', ascending=False).head(n)
    return top_n_df[['NCT_ID', 'Title', 'Cosine_Similarity']]


In [ ]:
# Test the search function with a sample query
query = 'Effects of Nifedipine on Congenital Adrenal Hyperplasia'
top_results = search_review(model, df, query, n=3)
print(top_results)

            NCT_ID                                              Title  \
26192  NCT00000102  Congenital Adrenal Hyperplasia: Calcium Channe...   
27676  NCT00001521  Three Drug Combination Therapy Versus Conventi...   
58695  NCT00519818  Comparison of Two Forms of Hydrocortisone in P...   

       Cosine_Similarity  
26192           0.677988  
27676           0.589365  
58695           0.563244  


In [ ]:
df["Brief_Summary"].iloc[26192]

'\n      This study will test the ability of extended release nifedipine (Procardia XL), a blood\r\n      pressure medication, to permit a decrease in the dose of glucocorticoid medication children\r\n      take to treat congenital adrenal hyperplasia (CAH).\r\n    '

In [ ]:
query = 'skin cancer'
top_results = search_review(model, df, query, n=3)
print(top_results)

            NCT_ID                                              Title  \
42708  NCT00390351  Investigate the Diagnostic Value and Possibili...   
48137  NCT00765193  The Impact of Total Body Skin Examination on S...   
37762  NCT00847912  CSP #562 - The VA Keratinocyte Carcinoma Chemo...   

       Cosine_Similarity  
42708           0.596394  
48137           0.594435  
37762           0.590338  


In [ ]:
df["Brief_Summary"].iloc[42708]

'\n      We will do systematic and functional assessment of non-melanoma skin cancer including\r\n      precancerous lesions by OCT imaging and other diagnostic methods. The systematic assessment\r\n      will be comparison of clinical diagnosis, histopathology, OCT images, spectrophotometry and\r\n      high frequency ultrasound of non-melanoma skin cancer Functional exploration of non-melanoma\r\n      skin cancer is done with Doppler-OCT and polarization-sensitive-OCT.\r\n    '

In [ ]:
query = 'Safety and effectiveness of MNTX for treating opioid-induced constipation in patients after orthopedic surgery'
top_results = search_review(model, df, query, n=3)
print(top_results)

            NCT_ID                                              Title  \
62780  NCT00402038  Study of Methylnaltrexone (MNTX) for the Relie...   
65236  NCT00949377  Can Methylnaltrexone Safely Treat Opioid Relat...   
62895  NCT00401362  A a Single Dose Efficacy Study in Inducing Lax...   

       Cosine_Similarity  
62780           0.804544  
65236           0.787409  
62895           0.753972  


In [ ]:
df["Brief_Summary"].iloc[62780]

'\n      To determine the efficacy of subcutaneous MNTX to relieve opioid-induced constipation in\r\n      patients with advanced medical illness.\r\n    '

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_metrics(df, query, n=30, threshold=0.5):
    # Generate the query embedding
    query_embedding = model.encode([query])
    query_embedding = query_embedding.reshape(1, -1)

    # Calculate cosine similarity
    df['Cosine_Similarity'] = df['Embedding_Vector'].apply(
    lambda x: cosine_similarity(query_embedding, np.array(x).reshape(1, -1))[0][0]
    )

    # Sort by similarity and get top 'n' results
    top_n_df = df.sort_values(by='Cosine_Similarity', ascending=False).head(n)

    # Simulate relevance : 1 if similarity above threshold, else 0
    top_n_df['Relevance'] = top_n_df['Cosine_Similarity'].apply(lambda x: 1 if x >= threshold else 0)

    # Precision@30
    precision_at_30 = top_n_df['Relevance'].head(30).mean()

    # Recall@30
    actual_relevant = top_n_df['Relevance'].sum()
    recall_at_30 = top_n_df['Relevance'].head(30).sum() / actual_relevant if actual_relevant else 0

    # F1-Score
    f1 = f1_score([1] * actual_relevant, top_n_df['Relevance'].head(actual_relevant).tolist()) if actual_relevant else 0

    # MAP: Calculate precision at each relevant position and average
    avg_precision = 0
    relevant_count = 0
    for i, relevance in enumerate(top_n_df['Relevance'], start=1):
        if relevance == 1:
            relevant_count += 1
            avg_precision += relevant_count / i
    map_score = avg_precision / relevant_count if relevant_count else 0

    # Display metrics
    print(f"Precision@30: {precision_at_30:.4f}")
    print(f"Recall@30: {recall_at_30:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"MAP: {map_score:.4f}")

# Example of usage
query = 'Effects of Nifedipine on Congenital Adrenal Hyperplasia'
calculate_metrics(df, query)

Precision@30: 0.3333
Recall@30: 1.0000
F1-score: 1.0000
MAP: 1.0000
